In [1]:
import pandas as pd
import os
import numpy as np
from expetator.tools import read_experiment, show_heatmap, add_objectives
#from expetator.tools import prune_vectors, mojitos_to_vectors, show_pct_distribution
from expetator.monitors import mojitos
import matplotlib.pyplot as plt

### Notes
- Nancy/gros & graphite no power
- Lyon/nova

In [2]:
PATH = "../../../dataset"
from scipy.stats import trim_mean
def mojitos_to_vectors_thang(mojitos_data, knowledge, proportiontocut=.1):
    tmp = [pd.DataFrame([trim_mean(bench_run, proportiontocut=proportiontocut)],
                          columns = bench_run.columns) for bench_run in mojitos_data]
    tmp = pd.concat(tmp, ignore_index=True)
    tmp['freq'] = knowledge.fmax
    return tmp

In [3]:
def loadDataset(paths):
    #load all files
    basename = []
    for path in paths:
        if path.endswith('.zip'):
            basename.append(path)
        else:
            for file in os.listdir(path):
                if os.path.isfile(os.path.join(path,file)) and file != '.DS_Store'\
                    and os.stat(os.path.join(path,file)).st_size > 1000:
                    basename.append(os.path.join(path,file))
    #print(basename)
    #read dataset
    knowledge, mojitos_data = read_experiment(basename, only_complete=True)
    #drop row where num of columns < 46: #{7, 28, 46, 48, 50}
    numcolslist = [len(a.columns) for a in mojitos_data]
    
    knowledge = knowledge.drop(np.where(np.array(numcolslist) < 46)[0]).reset_index(drop=True)
    for i in np.where(np.array(numcolslist) < 46)[0][::-1]:
        mojitos_data.pop(i)
    
    vectors = mojitos_to_vectors_thang(mojitos_data, knowledge)
    #drop experiments have inf/ 0 in power
    exper = (set(knowledge.expe) - set(knowledge[knowledge.power == float('inf')].expe)) - set(knowledge[knowledge.power == 0].expe)
    
    knowledge = knowledge[knowledge.expe.isin(exper)]
    vectors = vectors[vectors.index.isin(knowledge.index)].reset_index(drop=True)
    knowledge = knowledge.reset_index(drop=True)
    
    #create targets
    knowledge_metric = []
    vectors_metric = []
    for metric in ['energy', 'duration', 'etp']:
        knowledge_temp = knowledge.copy()
        knowledge_temp['metric'] = metric
        _ = add_objectives(knowledge_temp, objective_name='target', group_label='expe', metric=metric, ratio=1.05, stay_is_zero=False)
        _ = add_objectives(knowledge_temp, objective_name='targetZ', group_label='expe', metric=metric, ratio=1.05, stay_is_zero=True)
        knowledge_metric.append(knowledge_temp)
        vectors_metric.append(vectors)
    knowledge_metric = pd.concat(knowledge_metric, ignore_index=True)
    vectors_metric = pd.concat(vectors_metric, ignore_index=True)
    return knowledge_metric, vectors_metric

In [4]:
paths = [PATH+name for name in ['/nancy/grisou-2-11-20','/nancy/grimoire-2-11-20','/nancy/grisou-10-11-20','/nancy/grimoire-10-11-20',
                                '/lyon/taurus-2-11-20', '/lyon/taurus-10-11-20',
                                '/rennes/paravance-10-11-20'
                               ]]
paths.extend(['../../datasets/large/0_allpct_grisou-15.nancy.grid5000.fr_1589459642.zip',
 '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589565839.zip',
 '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589554681.zip',
 '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589492304.zip',
 '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589456512.zip'])
knowledge,vectors = loadDataset(paths)

uncompleted file: ../../../dataset/nancy/grisou-2-11-20/0_allpct_grisou-6.nancy.grid5000.fr_1603106978
uncompleted file: ../../../dataset/nancy/grisou-2-11-20/0_allpct_grisou-2.nancy.grid5000.fr_1603113128
uncompleted file: ../../../dataset/nancy/grisou-2-11-20/0_allpct_grisou-2.nancy.grid5000.fr_1603111177
uncompleted file: ../../../dataset/lyon/taurus-2-11-20/0_allpct_taurus-11.lyon.grid5000.fr_1603740888
uncompleted file: ../../../dataset/lyon/taurus-10-11-20/0_allpct_taurus-10.lyon.grid5000.fr_1604357071
error _data[0][1]) <= 4 ../../../dataset/nancy/grisou-2-11-20/0_allpct_grisou-35.nancy.grid5000.fr_1603067789_power/grisou-35.nancy.grid5000.fr_sp-C-64_1603067913
error _data[0][1]) <= 4 ../../../dataset/nancy/grisou-2-11-20/0_allpct_grisou-23.nancy.grid5000.fr_1603067789_power/grisou-23.nancy.grid5000.fr_ft-C-64_1603067893
no file: ../../../dataset/nancy/grisou-2-11-20/0_allpct_grisou-31.nancy.grid5000.fr_1603106994_power/grisou-31.nancy.grid5000.fr_sp-C-64_1603110540
error _data[

error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-42.rennes.grid5000.fr_1604351738_power/paravance-42.rennes.grid5000.fr_mpigeneric-16-0-0-0_1604360789
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-42.rennes.grid5000.fr_1604351738_power/paravance-42.rennes.grid5000.fr_bt-C-64_1604365721
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-42.rennes.grid5000.fr_1604351738_power/paravance-42.rennes.grid5000.fr_mg-D-64_1604366726
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-62.rennes.grid5000.fr_1604351720_power/paravance-62.rennes.grid5000.fr_mpigeneric-0-16-0-0_1604355034
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-62.rennes.grid5000.fr_1604351720_power/paravance-62.rennes.grid5000.fr_mpigeneric-0-16-0-0_1604355149
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravanc

error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-14.rennes.grid5000.fr_1604383502_power/paravance-14.rennes.grid5000.fr_sp-C-64_1604383918
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-14.rennes.grid5000.fr_1604383502_power/paravance-14.rennes.grid5000.fr_sp-C-64_1604384024
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-14.rennes.grid5000.fr_1604383502_power/paravance-14.rennes.grid5000.fr_mpigeneric-16-0-0-0_1604385195
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-14.rennes.grid5000.fr_1604383502_power/paravance-14.rennes.grid5000.fr_mpigeneric-0-16-0-0_1604385601
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-14.rennes.grid5000.fr_1604383502_power/paravance-14.rennes.grid5000.fr_mpigeneric-0-16-0-0_1604385666
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravanc

error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-67.rennes.grid5000.fr_1604351719_power/paravance-67.rennes.grid5000.fr_mpigeneric-0-0-8-8_1604355693
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-67.rennes.grid5000.fr_1604351719_power/paravance-67.rennes.grid5000.fr_mpigeneric-0-0-8-8_1604356029
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-67.rennes.grid5000.fr_1604351719_power/paravance-67.rennes.grid5000.fr_ft-C-64_1604366734
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-67.rennes.grid5000.fr_1604351719_power/paravance-67.rennes.grid5000.fr_ft-C-64_1604366765
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-5.rennes.grid5000.fr_1604351738_power/paravance-5.rennes.grid5000.fr_is-D-64_1604353441
error _data[0][1]) <= 4 ../../../dataset/rennes/paravance-10-11-20/0_allpct_paravance-5.rennes.grid5

In [5]:
knowledge.to_csv("knowledge_allmetrics.csv", index=False)

In [6]:
vectors.to_csv("vectors_allmetrics.csv", index=False)

In [7]:
np.sum(knowledge.fmax != vectors.freq)

0